In [32]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import iris_utils #TODO remove
import ipywidgets as widgets
from IPython.display import display
from meshcat import Visualizer

In [34]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer
from sandbox import rrtiris

In [35]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
vis = Visualizer(zmq_url=zmq_url)
vis.delete()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7054/static/


In [36]:

def load_model():
    model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
    #schunk = FindResourceOrThrow("drake/sos_iris_certifier/schunk_wsg_50_block.sdf")
    box_file_1 = FindResourceOrThrow("drake/sos_iris_certifier/assets/shelves.sdf")
    #box_file_2 = FindResourceOrThrow("drake/sos_iris_certifier/assets/shelves2.sdf")
    #box_file_3 = FindResourceOrThrow("drake/sos_iris_certifier/assets/shelves3.sdf")
    #box_file_4 = FindResourceOrThrow("drake/sos_iris_certifier/assets/shelves4.sdf")
    lid_file = FindResourceOrThrow("drake/sos_iris_certifier/assets/lid.urdf")
    ground_file = FindResourceOrThrow("drake/sos_iris_certifier/assets/ground.urdf")

    models =[]

    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

    parser = Parser(plant, scene_graph)
    #parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
    #        "drake/manipulation/models/wsg_50_description/package.xml")))
    models.append(parser.AddModelFromFile(model_file))
    models.append(parser.AddModelFromFile(box_file_1))
    #models.append(parser.AddModelFromFile(box_file_2))
    #models.append(parser.AddModelFromFile(box_file_3))
    #models.append(parser.AddModelFromFile(box_file_4))
    models.append(parser.AddModelFromFile(lid_file))
    models.append(parser.AddModelFromFile(ground_file))
    #models.append(parser.AddModelFromFile(schunk))

    sp = 0.4
    x_fac = 1.2
    locs = [ [0,0,0], 
            [x_fac*sp, 1.4*sp,0.4], [x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,1.4*sp,0.4], 
            [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
    
    locs = [locs[idx] for idx in [0,1,5,6]]
    
    idx = 0
    #for model in models[-1]:
    for model in models:
        plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), RigidTransform(locs[idx]))
        idx +=1
    
    #fuse S_C_H_U_N_K
    #plant.WeldFrames(plant.get_body(plant.GetBodyIndices(models[0])[-1]).body_frame(),
    #                 plant.get_body(plant.GetBodyIndices(models[-1])[0]).body_frame(),
    #                 RigidTransform(RollPitchYaw([np.pi/2,0, np.pi/2]).ToRotationMatrix(),[0,0,0.114]))

    plant.Finalize()
    return plant, builder, scene_graph, models

def load_vis(plant, builder, scene_graph, zmq_url, col_geom = 0):
    vis = Visualizer(zmq_url=zmq_url)
    vis.delete()
    if col_geom == 0:
        viz_role = Role.kIllustration
    else:
        viz_role = Role.kProximity
    visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, role=viz_role )

    diagram = builder.Build()
    visualizer.load()
    return vis, diagram
plant, builder, scene_graph, models = load_model()
vis, diagram = load_vis(plant, builder, scene_graph, zmq_url, col_geom = 1)
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
sg_context = scene_graph.GetMyContextFromRoot(context)
diagram.Publish(context)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7054/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6054...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7054/static/
Connected to meshcat-server.


In [37]:
#filter_manager = scene_graph.collision_filter_manager(sg_context)
inspector = scene_graph.model_inspector()

#pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
#print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

all_gids = []
for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity):
    #print(gid)
    name = inspector.GetName(gid)
    if "iiwa7::iiwa_link_1" in name:
        print(name)
        link_1 = gid
    if "iiwa7::iiwa_link_0" in name:
        print(name)
        link_0 = gid    
    all_gids.append(gid)
    #gid_name = inspector.GetName(gid)
    
one_zero_var_links = GeometrySet([link_0, link_1])
all_gid_set = GeometrySet(all_gids)
scene_graph.collision_filter_manager().Apply(CollisionFilterDeclaration().ExcludeBetween(one_zero_var_links, all_gid_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

iiwa7::iiwa_link_1_collision
iiwa7::iiwa_link_0_collision
58 -> 77


In [38]:
import ipywidgets as widgets
from IPython.display import display
q_slider = np.zeros((7,))
sliders = []
for low, up, joint_idx in zip(plant.GetPositionLowerLimits()[:7], plant.GetPositionUpperLimits()[:7], range(7)):
    sliders.append(
        widgets.FloatSlider(
        min=low, 
        max=up, 
        value=0, 
        description=f'iiwa_joint_{joint_idx}'
        )
    )


def handle_slider_change(change, idx):
    q_slider[idx] = change['new']
    #print(q, end="\r")
    plant.SetPositions(plant_context, q_slider)
    diagram.Publish(context)
    

for idx, slider in enumerate(sliders):
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='iiwa_joint_0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='iiwa_joint_1', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='iiwa_joint_2', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='iiwa_joint_3', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='iiwa_joint_4', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='iiwa_joint_5', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='iiwa_joint_6', max=3.05433, min=-3.05433)

In [39]:
display(vis.jupyter_cell())

In [40]:
start = np.array([1.43, -0.69, 0.83, 1.01, -1.97, 1.11, np.pi/2])
target = np.array([ 1.63294,  1.3056 ,  0.43   , -1.19   , -1.76706, -1.0944 ,
       -2.45433])
plant.SetPositions(plant_context, start)
diagram.Publish(context)

In [41]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 0.8e-5
iris_options.max_faces_per_collision_pair = 10
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.01
iris_options.enable_ibex = False
iris_options.q_star = np.zeros(7)
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
rational_fk = RationalForwardKinematics(plant)
poi = []
for pt in [start, target]:
    poi.append(rational_fk.ComputeTValue(pt, np.zeros((7,)) ))

#compute limits in t-space
limits_t = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_t.append(rational_fk.ComputeTValue(np.array(q), np.zeros((7,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])

def iris_handle(seed, domain):
    seed_q = rational_fk.ComputeQValue(np.array(seed), np.zeros((7,)) )
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    return IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)

In [44]:
region = iris_handle(poi[0], starting_poly)

In [45]:
regions = [region.ReduceInequalities()]

In [46]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [47]:
FindEpsilonLower(-0.3*np.ones((regions[0].A().shape[0],1)), 10.0*np.ones((regions[0].A().shape[0],1)), region.A(), region.b())

SystemExit: Failure at bazel-out/k8-opt/bin/solvers/_virtual_includes/constraint/drake/solvers/constraint.h:564 in LinearConstraint(): condition 'A_.array().isFinite().all()' failed.

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [31]:
%tb 

SystemExit: Failure at bazel-out/k8-opt/bin/solvers/_virtual_includes/constraint/drake/solvers/constraint.h:564 in LinearConstraint(): condition 'A_.array().isFinite().all()' failed.

In [31]:
edited_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 10
binary_search_options.epsilon_min = -0.0644
binary_search_options.max_iters = 5
binary_search_options.search_d = True
bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-3
bilinear_alternation_option.polytope_backoff_scale = 1e-5

solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)
for r in regions: 
    try:
        d_feasible = \
            cspace_free_region.CspacePolytopeBinarySearch(
                iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
                binary_search_options, solver_options)
        
        #C_final, d_final, P_final, q_final = \
        #    cspace_free_region.CspacePolytopeBilinearAlternation(
        #        iris_options.q_star, filtered_collision_pairs, r.A(), d_feasible,
        #       bilinear_alternation_option, solver_options)
        #C_final = np.vstack([C_final, P_joint_limits.A()])
        #d_final = np.concatenate([d_final, P_joint_limits.b()])
        #editted_regions.append(HPolyhedron(C_final, d_final))
    except Exception as e:
        print(e)
print(len(edited_regions))

[2022-01-23 17:19:06.140] [console] [info] Solver time 67.12762308120728
[2022-01-23 17:30:29.969] [console] [info] Solver time 282.63026118278503


binary search: the initial epsilon -0.0644 is infeasible
0


In [27]:
C = regions[0].MaximumVolumeInscribedEllipsoid().A()
radii2, R  =  np.linalg.eig(C.T @ C)
radii = np.sqrt(radii2)

In [33]:
region.ReduceInequalities


TypeError: 'pydrake.geometry.optimization.HPolyhedron' object is not subscriptable

# RRTIRIS SEEDING

In [20]:
plant.SetPositions(plant_context, start)
diagram.Publish(context)

In [21]:
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)

def eval_cons(q, c, tol):
    return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

In [22]:
RRTIRIS = rrtiris.RRTIRIS(poi[0],
                          poi[1],
                          limits = limits_t,
                          default_iris_domain = starting_poly,
                          iris_handle = iris_handle,
                          offset_scaler = 0.8,
                          init_goal_sample_rate = 0.05,
                          goal_sample_rate_scaler = 0.1,
                          verbose = True,
                          plotcallback = None,
                          )
RRTIRIS.run(1)

0
[RRT IRIS] Pos_samp [  4.95822984   0.89264185  -2.46051905   0.93647177   0.90372506
  -0.64562155 -13.96607497]  seed  [ 0.07178442 -0.1183294   0.84153307  1.39983734 -0.71316456  0.57561463
 -2.35743324]  dist  13.224370692880166
1.952428675231686
[RRT IRIS] it: 0 distance to target:  1.952 goalsample prob:  0.305


(False,
  <pydrake.geometry.optimization.Hyperellipsoid at 0x7fee2777a9b0>])

In [13]:
regions = RRTIRIS.node_regions

In [23]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [24]:
editted_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 10
binary_search_options.epsilon_min = -.2
binary_search_options.max_iters = 5
binary_search_options.search_d = True

bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-3
bilinear_alternation_option.polytope_backoff_scale = 1e-5

solver_options = mp.SolverOptions()
for r in regions: 
    try:
        d_feasible = \
            cspace_free_region.CspacePolytopeBinarySearch(
                iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
                binary_search_options, solver_options)
        C_final, d_final, P_final, q_final = \
            cspace_free_region.CspacePolytopeBilinearAlternation(
                iris_options.q_star, filtered_collision_pairs, r.A(), d_feasible,
                bilinear_alternation_option, solver_options)
        C_final = np.vstack([C_final, P_joint_limits.A()])
        d_final = np.concatenate([d_final, P_joint_limits.b()])
        editted_regions.append(HPolyhedron(C_final, d_final))
    except Exception as e:
        print(e)
print(len(editted_regions))

GenerateMonomialBasisWithOrderUpToOneHelper: Shouldn't have an empty input t_angles.
GenerateMonomialBasisWithOrderUpToOneHelper: Shouldn't have an empty input t_angles.
0
